In [1]:
import logging
logging.basicConfig()

In [2]:
from kazoo.client import KazooClient 

In [3]:
zk = KazooClient()
zk.start()

## Basic commands (getting and creating nodes)


In [4]:
 # Same as `ls /`
zk.get_children("/")

['zookeeper']

In [5]:
zk.get_children("/zookeeper"), zk.get_children("/zookeeper/quota") 

(['quota'], [])

In [6]:
zk.create(path="/mynode", value=b"first_version") 

'/mynode'

In [7]:
zk.get_children("/") 

['mynode', 'zookeeper']

In [8]:
zk.get('/mynode')  # Returns tuple (Value, Metadata), same as get /mynode && stat /mynode 

(b'first_version',
 ZnodeStat(czxid=33387, mzxid=33387, ctime=1640047375911, mtime=1640047375911, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=33387))

In [9]:
zk.set('/mynode', value=b"second_version") 

ZnodeStat(czxid=33387, mzxid=33388, ctime=1640047375911, mtime=1640047375992, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=14, numChildren=0, pzxid=33387)

In [10]:
zk.get('/mynode')  

(b'second_version',
 ZnodeStat(czxid=33387, mzxid=33388, ctime=1640047375911, mtime=1640047375992, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=14, numChildren=0, pzxid=33387))

## Creating two sequential nodes

In [11]:
zk.create('/mynode/child', value=b'im_sequential', sequence=True)
zk.create('/mynode/child', value=b'im_sequential_too', sequence=True)

'/mynode/child0000000001'

In [12]:
zk.get_children('/mynode') 

['child0000000000', 'child0000000001']

## Simultanious connections and Ephemeral nodes


In [13]:
zk1, zk2 = KazooClient(), KazooClient()
zk1.start()
zk2.start()

In [14]:
zk.create('/mygroup', b'top_node') 

'/mygroup'

In [15]:
zk1.create('/mygroup/grue', b'im_grue', ephemeral=True)
zk2.create('/mygroup/bleen', b'im_bleen', ephemeral=True)

'/mygroup/bleen'

In [16]:
zk.get_children('/mygroup') 

['grue', 'bleen']

In [17]:
zk1.get('/mygroup/bleen') 

(b'im_bleen',
 ZnodeStat(czxid=33395, mzxid=33395, ctime=1640047376213, mtime=1640047376213, version=0, cversion=0, aversion=0, ephemeralOwner=72057623493148867, dataLength=8, numChildren=0, pzxid=33395))

In [18]:
zk2.stop() 

In [19]:
zk.get_children('/mygroup') 

['grue']

In [20]:
try:
    zk1.get('/mygroup/bleen') 
except Exception as e:
    print('ERROR:', type(e))

ERROR: <class 'kazoo.exceptions.NoNodeError'>


In [21]:
 zk1.stop(), zk2.stop()

(None, None)

In [22]:
 zk.delete('/mygroup', recursive=True)

## Watching events

In [23]:
zk.create('/myconfig', b'sheep_count=1') 

'/myconfig'

In [24]:
zk1 = KazooClient()
zk1.start()

In [25]:
 def watch_node(state):
    print(state)

In [26]:
zk1.get('/myconfig', watch=watch_node) 

(b'sheep_count=1',
 ZnodeStat(czxid=33399, mzxid=33399, ctime=1640047376454, mtime=1640047376454, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=33399))

In [27]:
zk.set('/myconfig', b'sheep_count=2')

WatchedEvent(type='CHANGED', state='CONNECTED', path='/myconfig')

ZnodeStat(czxid=33399, mzxid=33401, ctime=1640047376454, mtime=1640047376559, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=33399)

In [28]:
zk.set('/myconfig', b'sheep_count=3')  

ZnodeStat(czxid=33399, mzxid=33402, ctime=1640047376454, mtime=1640047376590, version=2, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=33399)

CLI commands

In [29]:
# clearing all trash in the end
for path in ['mynode', 'myconfig']:
    zk.delete(f'/{path}', recursive=True)

In [30]:
zk.get_children("/")

['zookeeper']

In [31]:
zk.stop()
zk.close()